In [1]:
import pika
import sys
pika.__version__

'1.1.0'

In [2]:
#!sudo service rabbitmq-server restart
#!sudo rabbitmqctl status

In [3]:
!sudo rabbitmqctl list_users

Listing users ...
guest	[administrator]
myguest	[]


In [4]:
!sudo rabbitmqctl list_bindings

Listing bindings ...
	exchange	amq.gen-nb9coprYp33EVzYkbzqHMA	queue	amq.gen-nb9coprYp33EVzYkbzqHMA	[]
	exchange	amq.gen-s3n1Dhhem2Z9GN9WCTyDxg	queue	amq.gen-s3n1Dhhem2Z9GN9WCTyDxg	[]
	exchange	hello2	queue	hello2	[]
	exchange	rpc_queue	queue	rpc_queue	[]
	exchange	task_queue	queue	task_queue	[]
direct_logs	exchange	task_queue	queue	NYC	[]
direct_logs	exchange	task_queue	queue	Pittsburgh	[]
direct_logs	exchange	task_queue	queue	Washington	[]
ping	exchange	amq.gen-nb9coprYp33EVzYkbzqHMA	queue	amq.gen-nb9coprYp33EVzYkbzqHMA	[]


In [5]:
credentials = pika.PlainCredentials('myguest', 'myguestpwd')
    
#URL='localhost'
URL='PROD-JOB-844fd7d2202ac4da.elb.us-east-2.amazonaws.com'
parameters =  pika.ConnectionParameters(URL, port=5672, credentials=credentials)

exchangeName = 'ping'

In [6]:
def on_response(self, ch, method, props, body):
    print(body)

In [8]:
connection = pika.BlockingConnection(parameters)
channel = connection.channel()

channel.exchange_declare(exchange=exchangeName, exchange_type='fanout')

#ping all workers
#for fanout type exchange, routing key/queue is useless
properties = pika.spec.BasicProperties(content_type = "text/plain", delivery_mode = 1)
message = "Ping EveryOne!"

#channel.basic_publish(exchange = exchangeName, routing_key = "", body = message, properties = properties)


myqueue = channel.queue_declare('')
print("myqueue:", myqueue)
callback_queue = myqueue.method.queue #The queue we just created

    
#disconnect
#connection.close()

myqueue: <METHOD(['channel_number=1', 'frame_type=1', "method=<Queue.DeclareOk(['consumer_count=0', 'message_count=0', 'queue=amq.gen-JlwenA-ZMVn2o18wljndww'])>"])>


'ctag1.865614f89ec04cf1aca1b8e9ebff79bf'

In [9]:

channel.basic_publish(
            exchange=exchangeName,
            routing_key="",
            properties=pika.BasicProperties(
                reply_to=callback_queue,
            ),
            body=message)

In [10]:
channel.basic_consume(
            queue=callback_queue, #the queue we created above
            on_message_callback=on_response, #The callback function to run when we receive a message from the workers
            auto_ack=True)


'ctag1.012edfcf04224003a390b45fd591f9d0'

**After this, we should see that all workers received the same message**

In [7]:
!sudo rabbitmqctl list_bindings

Listing bindings ...
	exchange	amq.gen-gf_WYyWCqa0YVkH8hbihPA	queue	amq.gen-gf_WYyWCqa0YVkH8hbihPA	[]
	exchange	hello2	queue	hello2	[]
	exchange	rpc_queue	queue	rpc_queue	[]
	exchange	task_queue	queue	task_queue	[]
direct_logs	exchange	task_queue	queue	NYC	[]
direct_logs	exchange	task_queue	queue	Pittsburgh	[]
direct_logs	exchange	task_queue	queue	Washington	[]
ping	exchange	amq.gen-gf_WYyWCqa0YVkH8hbihPA	queue	amq.gen-gf_WYyWCqa0YVkH8hbihPA	[]


In [8]:
!sudo rabbitmqctl list_exchanges

Listing exchanges ...
	direct
amq.direct	direct
amq.fanout	fanout
amq.headers	headers
amq.match	headers
amq.rabbitmq.log	topic
amq.rabbitmq.trace	topic
amq.topic	topic
direct_logs	direct
logs	fanout
ping	fanout
